In [ ]:
# color list ranked by pairwise distance (generated by https://mokole.com/palette.html)
color_list = [
    (47, 79, 79),
    (85, 107, 47),
    (139, 69, 19),
    (34, 139, 34),
    (72, 61, 139),
    (184, 134, 11),
    (70, 130, 180),
    (0, 0, 128),
    (127, 0, 127),
    (143, 188, 143),
    (176, 48, 96),
    (255, 69, 0),
    (255, 255, 0),
    (0, 255, 0),
    (138, 43, 226),
    (0, 255, 127),
    (220, 20, 60),
    (0, 255, 255),
    (0, 0, 255),
    (173, 255, 47),
    (218, 112, 214),
    (255, 127, 80),
    (255, 0, 255),
    (30, 144, 255),
    (144, 238, 144),
    (173, 216, 230),
    (255, 20, 147),
    (123, 104, 238),
    (255, 222, 173),
    (255, 192, 203),   
]

In [ ]:
import rdflib

from ipyradiant import FileManager, PathLoader
from ipyradiant.visualization import CytoscapeViewer

In [ ]:
lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
rdf_graph = lw.graph
qres = lw.graph.query(
    """
    PREFIX planet: <https://swapi.co/resource/planet/>
    PREFIX human: <https://swapi.co/resource/human/>
    PREFIX droid: <https://swapi.co/resource/droid/>
    PREFIX gungan: <https://swapi.co/resource/gungan/>
    PREFIX film: <https://swapi.co/resource/film/>
    
    CONSTRUCT {
        ?s ?p ?o .
    }
    WHERE {
        ?s ?p ?o .
        
        VALUES (?s) {
            (human:1)
            (human:4)
            (human:5)
            (human:14)
            (human:22)
            (human:67)
            (human:69)
            (human:85)
            (human:88)
            (gungan:36)
            (droid:2)
            (droid:3)
            (droid:23)
            (droid:87)
            (film:1)
            (film:2)
            (film:3)
            (film:4)
            (film:5)
            (film:6)
            (film:7)
            (planet:1)
            (planet:2)
            (planet:3)
            (planet:4)
            (planet:5)
            (planet:6)
            (planet:7)
        }
    }
    """
)

simple_graph = rdflib.graph.Graph().parse(data=qres.serialize(format="xml"))

In [ ]:
cv = CytoscapeViewer(animate=True, node_labels=True, edge_labels=True)
cv.graph = simple_graph
# a little easier to see
cv.cytoscape_widget.layout.height = "80vh"

In [ ]:
cv

In [ ]:
# get types (in order to color-code)
from ipyradiant.query.api import SPARQLQueryFramer

In [ ]:
# note: need to make this available in the class for users to extend based on their unique predicates for type specification
class TypeCount(SPARQLQueryFramer):
    sparql = """
    SELECT DISTINCT ?type_ (COUNT(?node) AS ?count)
    WHERE {
        ?node a ?type_ .
    }
    GROUP BY ?type_
    """
    
type_count = TypeCount.run_query(simple_graph).sort_values(by=["count"], ascending=False)
types = set(type_count.type_)

In [ ]:
type_count

In [ ]:
import ipywidgets as W
from ipyradiant.basic_tools.custom_uri_ref import CustomURIRef


def get_desc(uri, namespaces, count=None):
    shorthand = str(CustomURIRef(uri, namespaces=namespaces))
    if count:
        return f"{shorthand}  [{count}]"

In [ ]:
# Has to be a multi-select so that we can get the URI from the CustomURIRef class
# How to we auto-construct classes for the types/predicates, and assign nodes in the cytoscape graph to that class?

In [ ]:
# map type URIs to their css class name for ipycytoscape
uri_to_string_type = {
    uri: str(CustomURIRef(uri, namespaces=simple_graph.namespace_manager)).replace(":", "-")
    for uri in type_count.type_
}
uri_to_string_type["multi-type"] = "multi-type"
uri_to_string_type

In [ ]:
# convert to css format
color_list_css = [f"rgb({r},{g},{b})" for r, g, b in color_list]

# assign colors to css classes
assert len(uri_to_string_type.keys()) <= len(color_list), f"Cannot render more than {len(color_list)} visually disctinct colors."
color_type_map = list(zip([*uri_to_string_type.values(), "multi-type"], color_list_css))
color_type_map

In [ ]:
import rdflib

from ipyradiant.visualization.explore.interactive_exploration import add_cyto_class, remove_cyto_class

# assign CSS classes to nodes based on their rdf:type
for node in cv.cytoscape_widget.graph.nodes:
    node_types = node.data.get("rdf:type", [])
    if type(node_types) == rdflib.term.URIRef:
        node_types = (node_types,)
        
    if len(node_types) == 1:
        # assign specific class to node
        assert node_types[0] in uri_to_string_type
        css_class = uri_to_string_type[node_types[0]]
        node.data["type_"] = css_class
    else:
        # assign multiple classes to node and assign aggregate_type for coloring
#         for node_type in node_types:
#             assert node_type in uri_to_string_type
#             css_class = uri_to_string_type[node_type]
#             node.classes = add_cyto_class(node, css_class)
        node.data["type_"] = "multi-type"

> NOTE: classes have to match exactly. i.e. if the style is for classes='test' and the node classes are 'test other' the style __will not be applied__.

> NOTE: classes must be set on the node/edge __data__, e.g. `node.data['classes'] = "class1 class2"`

> NOTE: the above two statements are correct when using a data attribute `classes` i.e. node.data["classes"]. However, CSS classes are specified on the element directly, i.e. node.classes. These are composable.

In [ ]:
# use css data attribute style to color based on type
color_classes = []
for class_name, rgb_code in color_type_map:
    color_classes.append(
        {
            'selector': f"node[type_ = '{class_name}']",
            'style': {
                'background-color': f"{rgb_code}",
            }
        }
    )

In [ ]:
# visibility classes
invisible_node_class = {
    'selector': 'node.invisible',
    'style': {
        'visibility': 'hidden',
    }
}
invisible_edge_class = {
    'selector': 'edge.invisible',
    'style': {
        'visibility': 'hidden',
    }
}

# TODO
* [X] toggle visibility of edges
* [X] update to multiselect
* [X] use `visibility` on css (visibility in CSS)
* [X] update example to more nodes
* [X] investigate layout options and see if it changes
* [X] investigate node.classes vs node.data["classes"] for the selectors (ideally only use node.classes)
* [ ] color types
* colorpicker for types?
* move from SVG to canvas renderer?

In [ ]:
# change the cytoscape widget style
old_style = list(cv.cytoscape_widget.get_style())  # must be a copy!!!!!!
old_style.extend([*color_classes, invisible_node_class, invisible_edge_class])
cv.cytoscape_widget.set_style(old_style)

In [ ]:
# node_iri to node (for mapping to edges)
# TODO is there a way to enhance the adjacency matrix?
iri_to_node = {
    str(node.data["iri"]): node
    for node in cv.cytoscape_widget.graph.nodes
}

In [ ]:
from typing import Union
import ipycytoscape as cyto

from ipyradiant.visualization.explore.interactive_exploration import add_cyto_class, remove_cyto_class


def update_classes(change):
    """Updates the CSS classes for nodes/edges.
    
    TODO need to optimize so that we don't have to iterate through every node/edge
    """
    assert all([uri in uri_to_string_type for uri in change.new])
    visible_iris = set(change.new)
    
    for node in cv.cytoscape_widget.graph.nodes:
        raw_types = node.data["rdf:type"]
        types = raw_types if type(raw_types) is tuple else (raw_types,)
        if not any([_type in visible_iris for _type in types]):
            node.classes = 'invisible'
        else:
            node.classes = ''
            
    for edge in cv.cytoscape_widget.graph.edges:
        source_node = iri_to_node[edge.data["source"]]
        target_node = iri_to_node[edge.data["target"]]

        if "invisible" in source_node.classes or "invisible" in target_node.classes:
            edge.classes = 'invisible'
        else:
            edge.classes = 'directed'
    
    # update front-end (option 1 - jarring update)
    #cv.cytoscape_widget.relayout()
    
    # update front-end (option 2 - instant, trigger with replicated style)
    cv.cytoscape_widget.set_style(list(cv.cytoscape_widget.get_style()))

In [ ]:
ms_options = []
for uri, count in type_count.values:
    description = get_desc(uri, simple_graph.namespace_manager, count)
    ms_options.append((description, uri))
    
ms_ex = W.SelectMultiple(
    value=tuple(uri for _, uri in ms_options),
    options=ms_options,
    layout=W.Layout(
        border='solid 1px',
        flex='auto'
    )
)
ms_ex.observe(update_classes, "value")
ms_ex

In [ ]:
cv

In [ ]:
cv.cytoscape_widget.graph.nodes[0].data["type_"]

In [ ]:
cv.cytoscape_widget.get_style()